In [2]:
from __future__ import print_function
import cv2
import os
import glob
import numpy as np


import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
def load_data(categories):
    # Training and Testing
    data             = []
    target           = []
    size_image       = 48
    img_dir          = 'train'    
    
    for idx, category in enumerate(categories):
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            data.append(img_resize)
            target.append(idx)
    
    

    target     = keras.utils.to_categorical(target, len(categories))    
    data       = np.array(data)
    target     = np.array(target)
    
    data =data.astype(np.uint8)/255.0 
    
    
    stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
    for test_idx, train_idx in stratSplit.split(data, target):        
        train_data   = data[train_idx]
        train_target = target[train_idx]                        
        test_data    = data[test_idx]
        test_target  = target[test_idx]
        
    # Validation
    val_data   = []
    val_target = []
    img_dir    = 'val'
    for idx, category in enumerate(categories):        
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)        
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            val_data.append(img_resize)
            val_target.append(idx)
    
    val_target   = keras.utils.to_categorical(val_target, len(categories))                
    val_data     = np.array(val_data)
    
    val_data =val_data.astype(np.uint8)/255.0
    
    val_target   = np.array(val_target)
    
    np.random.shuffle(val_data)
    np.random.shuffle(val_target)    
    
    return train_data, train_target, test_data, test_target, val_data, val_target

In [11]:
batch_size        = 32
num_classes       = 7
save_dir          = os.path.join(os.getcwd(), 'saved_models')
model_name        = 'faces.h5'
categories  = ['angry','disgust','fear','happy','neutral','sad','surprise']
train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

In [12]:
epochs            = 100

In [9]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu', input_shape=train_data.shape[1:]))
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu' ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), metrics=['accuracy'])
# model.summary()

batch_size = 32
print(val_data.shape)
model.fit(train_data,train_target,epochs = epochs, batch_size = batch_size,validation_data = (val_data, val_target), shuffle = True)


(1000, 48, 48, 3)
Train on 2328 samples, validate on 1000 samples
Epoch 1/100
2328/2328 [==============================] - 50s 22ms/step - loss: 4.0147 - acc: 0.1731 - val_loss: 2.1606 - val_acc: 0.1380
Epoch 2/100
2328/2328 [==============================] - 44s 19ms/step - loss: 2.6194 - acc: 0.1783 - val_loss: 1.9096 - val_acc: 0.2370
Epoch 3/100
2328/2328 [==============================] - 44s 19ms/step - loss: 2.1988 - acc: 0.1821 - val_loss: 1.8671 - val_acc: 0.4010
Epoch 4/100
2328/2328 [==============================] - 44s 19ms/step - loss: 2.0300 - acc: 0.1980 - val_loss: 1.8700 - val_acc: 0.4660
Epoch 5/100
2328/2328 [==============================] - 45s 19ms/step - loss: 1.9599 - acc: 0.2083 - val_loss: 1.8045 - val_acc: 0.4640
Epoch 6/100
2328/2328 [==============================] - 44s 19ms/step - loss: 1.9371 - acc: 0.2212 - val_loss: 1.8574 - val_acc: 0.4650
Epoch 7/100
2328/2328 [==============================] - 45s 19ms/step - loss: 1.9254 - acc: 0.2131 - val_loss: 

Epoch 60/100
2328/2328 [==============================] - 48s 21ms/step - loss: 1.0599 - acc: 0.5790 - val_loss: 4.3746 - val_acc: 0.2620
Epoch 61/100
2328/2328 [==============================] - 56s 24ms/step - loss: 0.9862 - acc: 0.5919 - val_loss: 4.9120 - val_acc: 0.2780
Epoch 62/100
2328/2328 [==============================] - 50s 21ms/step - loss: 0.9812 - acc: 0.6138 - val_loss: 4.2295 - val_acc: 0.2340
Epoch 63/100
2328/2328 [==============================] - 49s 21ms/step - loss: 0.9671 - acc: 0.6001 - val_loss: 4.0973 - val_acc: 0.2250
Epoch 64/100
2328/2328 [==============================] - 50s 21ms/step - loss: 0.9505 - acc: 0.6057 - val_loss: 5.1668 - val_acc: 0.3050
Epoch 65/100
2328/2328 [==============================] - 54s 23ms/step - loss: 0.9348 - acc: 0.6211 - val_loss: 4.8592 - val_acc: 0.2300
Epoch 66/100
2328/2328 [==============================] - 56s 24ms/step - loss: 0.9716 - acc: 0.5975 - val_loss: 5.2829 - val_acc: 0.2570
Epoch 67/100
2328/2328 [==========

In [10]:
model_name        = 'facesv1.h5'
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

scores = model.evaluate(val_data, val_target, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Saved trained model at /home/alexander/Documents/Cursos/RN/Tareas/Projeto2/saved_models/facesv1.h5 
582/582 [==============================] - 4s 6ms/step
Test loss: 1.3722925921486007
Test accuracy: 0.6975945015133861
1000/1000 [==============================] - 6s 6ms/step
Val loss: 7.136528518676758
Val accuracy: 0.223


In [13]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=train_data.shape[1:], kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))    
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))    
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

learning_rate = 0.001
adam = Adam(lr = learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [14]:
batch_size = 32
print(val_data.shape)
model.fit(train_data,train_target,epochs = epochs, batch_size = batch_size,validation_data = (val_data, val_target), shuffle = True)

(1000, 48, 48, 3)
Train on 2328 samples, validate on 1000 samples
Epoch 1/100
2328/2328 [==============================] - 76s 33ms/step - loss: 3.5901 - acc: 0.1641 - val_loss: 2.0292 - val_acc: 0.1790
Epoch 2/100
2328/2328 [==============================] - 70s 30ms/step - loss: 2.3319 - acc: 0.1791 - val_loss: 1.9167 - val_acc: 0.4530
Epoch 3/100
2328/2328 [==============================] - 51s 22ms/step - loss: 2.1305 - acc: 0.1967 - val_loss: 1.9188 - val_acc: 0.4650
Epoch 4/100
2328/2328 [==============================] - 51s 22ms/step - loss: 2.0459 - acc: 0.2075 - val_loss: 1.8994 - val_acc: 0.4650
Epoch 5/100
2328/2328 [==============================] - 58s 25ms/step - loss: 2.0218 - acc: 0.2178 - val_loss: 1.8997 - val_acc: 0.4650
Epoch 6/100
2328/2328 [==============================] - 73s 31ms/step - loss: 1.9940 - acc: 0.2195 - val_loss: 1.8740 - val_acc: 0.4650
Epoch 7/100
2328/2328 [==============================] - 69s 29ms/step - loss: 1.9594 - acc: 0.2341 - val_loss: 

Epoch 60/100
2328/2328 [==============================] - 51s 22ms/step - loss: 1.8550 - acc: 0.2573 - val_loss: 1.7418 - val_acc: 0.4580
Epoch 61/100
2328/2328 [==============================] - 51s 22ms/step - loss: 1.8527 - acc: 0.2569 - val_loss: 1.7337 - val_acc: 0.4650
Epoch 62/100
2328/2328 [==============================] - 51s 22ms/step - loss: 1.8538 - acc: 0.2573 - val_loss: 1.7319 - val_acc: 0.4660
Epoch 63/100
2328/2328 [==============================] - 51s 22ms/step - loss: 1.8493 - acc: 0.2620 - val_loss: 1.7303 - val_acc: 0.4650
Epoch 64/100
2328/2328 [==============================] - 50s 22ms/step - loss: 1.8554 - acc: 0.2543 - val_loss: 1.7287 - val_acc: 0.4650
Epoch 65/100
2328/2328 [==============================] - 50s 22ms/step - loss: 1.8529 - acc: 0.2590 - val_loss: 1.7112 - val_acc: 0.4650
Epoch 66/100
2328/2328 [==============================] - 50s 21ms/step - loss: 1.8556 - acc: 0.2552 - val_loss: 1.7292 - val_acc: 0.4650
Epoch 67/100
2328/2328 [==========

In [15]:
model_name        = 'facesv2.h5'
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

scores = model.evaluate(val_data, val_target, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Saved trained model at /home/alexander/Documents/Cursos/RN/Tareas/Projeto2/saved_models/facesv2.h5 
582/582 [==============================] - 4s 6ms/step
Test loss: 1.8071630287825857
Test accuracy: 0.2920962199824782
1000/1000 [==============================] - 6s 6ms/step
Val loss: 1.743383279800415
Val accuracy: 0.424


In [6]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

datagen.fit(train_data)

# Fit the model on the batches generated by datagen.flow().
generator = datagen.flow(train_data, train_target,batch_size=batch_size)
model.fit_generator(generator, epochs=epochs,validation_data=(test_data, test_target),workers=4, steps_per_epoch = len(generator))

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


W1003 15:05:08.971750 19840 deprecation.py:323] From C:\Users\1312\.conda\envs\tf\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
73/73 [==============================] - 113s 2s/step - loss: 3.2465 - acc: 0.1727 - val_loss: 1.8979 - val_acc: 0.2509
Epoch 2/30
73/73 [==============================] - 106s 1s/step - loss: 2.3735 - acc: 0.1996 - val_loss: 1.9979 - val_acc: 0.1976
Epoch 3/30
73/73 [==============================] - 112s 2s/step - loss: 2.2985 - acc: 0.2021 - val_loss: 1.9815 - val_acc: 0.2199
Epoch 4/30
73/73 [==============================] - 107s 1s/step - loss: 2.0708 - acc: 0.2125 - val_loss: 1.8968 - val_acc: 0.2354
Epoch 5/30
73/73 [==============================] - 111s 2s/step - loss: 1.9698 - acc: 0.2125 - val_loss: 1.8810 - val_acc: 0.2526
Epoch 6/30
73/73 [==============================] - 110s 2s/step - loss: 1.9480 - acc: 0.2059 - val_loss: 1.8907 - val_acc: 0.2560
Epoch 7/30
49/73 [===================>..........] - ETA: 33s - loss: 1.9416 - acc: 0.2117

KeyboardInterrupt: 

In [30]:
batch_size = 32
print(val_data.shape)
model.fit(train_data,train_target,epochs = epochs, batch_size = batch_size,validation_data = (val_data, val_target), shuffle = True)

(1000, 48, 48, 3)
Train on 2328 samples, validate on 1000 samples
Epoch 1/30
2328/2328 [==============================] - 288s 124ms/step - loss: 2.3914 - acc: 0.1654 - val_loss: 1.9407 - val_acc: 0.3610
Epoch 2/30
 832/2328 [=========>....................] - ETA: 2:26 - loss: 2.3327 - acc: 0.1755

KeyboardInterrupt: 